<a href="https://colab.research.google.com/github/parsh-waah/Coursera_Capstone/blob/master/CoffeeShop_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Data Science Capstone by IBM/Coursera

### Table of contents 

1.   Introduction: Business Problem
2.   Data
3.   Methodology
4.   Result and Discussion
5.   Conclusion








## Introduction: Business Problem

In this project we will try to find an optimal location for a coffee shop. Specifically, this report will be targeted to stakeholders interested in opening a Coffee Shop in Toronto, Canada.

Since there are lots of cafes in Toronto we will try to detect locations that are not already crowded with cafes. We would also prefer locations are densely populated and have a high average income.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data

To provide the stakeholders the necessary information I'll be combining Toronto's 2016 Census that contains Population, Average income per Neighbourhood with Toronto's Neighbourhoods shapefile and Foursquare API to collect competitors on the same neighbourhoods.

Toronto's Census data is publicly available at this website: https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a

Toronto Neighbourhoods' shapefile is publicly available at this website: https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#a45bd45a-ede8-730e-1abc-93105b2c439f


In [ ]:
import pandas as pd

In [ ]:
df_toronto= pd.read_csv("neighbourhood_toronto.csv")

In [ ]:
df_toronto.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,...,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,...,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,...,NIA,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,NIA,NIA,NIA,No Designation,No Designation,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,NIA,NIA,No Designation,No Designation,NIA,No Designation,NIA,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526","27,695","15,873","25,797","21,396","13,154","23,236","6,577","29,960","22,291","21,737","7,727","14,257","9,266","11,499","17,757","11,669","9,955","10,968","13,362","31,340","26,984","16,472","15,935","14,133","9,666","17,180","27,051","25,003","36,625","35,052",...,"10,803","10,529","22,246","14,974","20,923","46,496","10,070","9,941","16,724","21,849","27,876","17,812","24,623","25,051","27,446","15,683","21,567","10,360","21,108","16,556","7,607","17,510","65,913","27,392","33,312","26,274","17,992","11,098","27,917","50,434","16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177","26,918","15,434","19,348","17,671","13,530","23,185","6,488","27,876","21,856","22,057","7,763","14,302","8,713","11,563","17,787","12,053","9,851","10,487","13,093","28,349","24,770","14,612","15,703","13,743","9,444","16,712","26,739","24,363","34,631","34,659",...,"10,007","10,488","22,267","15,050","20,631","45,912"

In [ ]:
df_toronto

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,...,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,...,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,...,NIA,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,NIA,NIA,NIA,No Designation,No Designation,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,NIA,NIA,No Designation,No Designation,NIA,No Designation,NIA,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526","27,695","15,873","25,797","21,396","13,154","23,236","6,577","29,960","22,291","21,737","7,727","14,257","9,266","11,499","17,757","11,669","9,955","10,968","13,362","31,340","26,984","16,472","15,935","14,133","9,666","17,180","27,051","25,003","36,625","35,052",...,"10,803","10,529","22,246","14,974","20,923","46,496","10,070","9,941","16,724","21,849","27,876","17,812","24,623","25,051","27,446","15,683","21,567","10,360","21,108","16,556","7,607","17,510","65,913","27,392","33,312","26,274","17,992","11,098","27,917","50,434","16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177","26,918","15,434","19,348","17,671","13,530","23,185","6,488","27,876","21,856","22,057","7,763","14,302","8,713","11,563","17,787","12,053","9,851","10,487","13,093","28,349","24,770","14,612","15,703","13,743","9,444","16,712","26,739","24,363","34,631","34,659",...,"10,007","10,488","22,267","15,050","20,631","45,912"

In [ ]:
df_toronto=df_toronto.drop(['Topic','Data Source'],axis=1)

In [ ]:
class_23 = df_toronto[df_toronto["Characteristic"].isin(['Population, 2016','Population density per square kilometre','Youth (15-24 years)','Working Age (25-54 years)','Land area in square kilometres','Private dwellings occupied by usual residents','Total - Income statistics in 2015 for the population aged 15 years and over in private households'])]

In [ ]:
class_23

,_id,Category,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,...,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
2,3,Population,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526","27,695","15,873","25,797","21,396","13,154","23,236","6,577","29,960","22,291","21,737","7,727","14,257","9,266","11,499","17,757","11,669","9,955","10,968","13,362","31,340","26,984","16,472","15,935","14,133","9,666","17,180","27,051","25,003","36,625","35,052","11,785","21,381",...,"10,803","10,529","22,246","14,974","20,923","46,496","10,070","9,941","16,724","21,849","27,876","17,812","24,623","25,051","27,446","15,683","21,567","10,360","21,108","16,556","7,607","17,510","65,913","27,392","33,312","26,274","17,992","11,098","27,917","50,434","16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
6,7,Population,Private dwellings occupied by usual residents,"1,112,929","9,120","8,136","4,616","15,934","12,124","6,089","15,074","9,532","4,698","8,607","2,650","10,766","9,198","7,324","3,125","5,737","3,241","5,663","6,482","6,442","3,680","5,406","4,379","19,680","9,246","6,554","5,902","5,489","3,925","7,022","9,968","8,777","15,320","13,121","5,436","9,180",...,"4,960","3,846","9,055","6,887","10,054","13,389","3,818","3,650","5,923","11,390","12,073","6,428","7,819","10,377","10,063","6,272","9,428","3,280","7,123","6,910","3,393","7,412","40,756","9,986","10,285","10,234","7,600","4,226","10,106","22,304","7,549","8,509","18,436","5,454","3,449","5,887","5,676","7,012","10,170","5,344"
7,8,Population,Population density per square kilometre,"4,334","3,929","3,034","2,435","10,863","2,775","3,377","14,097","4,195","3,240","4,209","3,614","4,011","3,765","6,282","8,134","7,791","1,040","6,764","5,045","8,335","6,464","5,683","2,479","23,044","3,632","3,979","2,273","7,478","8,554","7,881","6,441","4,146","9,819","2,337","8,540","8,038",...,"16,880","4,229","4,414","9,851","4,500","1,260","6,333","4,734","5,395","9,583","3,136","2,430","5,436","3,199","5,073","15,528","6,058","3,130","6,787","9,570","5,395","3,710","8,943","2,856","1,117","7,339","7,197","7,601","2,724","10,087","5,820","4,007","4,345","7,838","6,722","8,541","7,162","10,708","2,086","2,451"
8,9,Population,Land area in square kilometres,630.2,7.41,7.83,4.95,2.81,9.98,4.7,1.83,5.1,4.06,5.52,1.82,7.47,5.92,3.46,0.95,1.83,8.91,1.7,3.52,1.4,1.54,1.93,5.39,1.36,7.43,4.14,7.01,1.89,1.13,2.18,4.2,6.03,3.73,15,1.38,2.66,...,0.64,2.49,5.04,1.52,4.65,36.89,1.59,2.1,3.1,2.28,8.89,7.33,4.53,7.83,5.41,1.01,3.56,3.31,3.11,1.73,1.41,4.72,7.37,9.59,29.81,3.58,2.5,1.46,10.25,5,2.91,5.53,12.31,1.6,1.17,1.68,1.65,1.17,13.23,6.04
10,11,Population,Youth (15-24 years),"340,270","3,705","3,360","1

In [ ]:
df_modified=class_23.transpose()

In [ ]:
df_modified

,2,6,7,8,10,11,944
_id,3,7,8,9,11,12,945
Category,Population,Population,Population,Population,Population,Population,Income
Characteristic,"Population, 2016",Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Youth (15-24 years),Working Age (25-54 years),Total - Income statistics in 2015 for the popu...
City of Toronto,"2,731,571","1,112,929","4,334",630.2,"340,270","1,229,555","2,294,785"
Agincourt North,"29,113","9,120","3,929",7.41,"3,705","11,305","25,005"
...,...,...,...,...,...,...,...
Wychwood,"14,349","5,887","8,541",1.68,"1,320","6,420","11,345"
Yonge-Eglinton,"11,817","5,676","7,162",1.65,"1,225","5,860","9,995"
Yonge-St.Clair,"12,528","7,012","10,708",1.17,920,"5,960","11,170"
York University Heights,"27,593","10,170","2,086",13.23,"4,750","12,290","23,530"


In [ ]:
df_modified.rename(columns={2: 'Population,2016',6:'Private dwellings occupied by usual residents',7:'Population density per square kilometre',8:'Land area in square kilometres',10:'Youth (15-24 years)',11:'Working Age (25-54 years)',944:'Total - Income statistics in 2015 for the population aged 15 years and older'}, inplace=True)

In [ ]:
df_modified

,"Population,2016",Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Youth (15-24 years),Working Age (25-54 years),Total - Income statistics in 2015 for the population aged 15 years and older
_id,3,7,8,9,11,12,945
Category,Population,Population,Population,Population,Population,Population,Income
Characteristic,"Population, 2016",Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Youth (15-24 years),Working Age (25-54 years),Total - Income statistics in 2015 for the popu...
City of Toronto,"2,731,571","1,112,929","4,334",630.2,"340,270","1,229,555","2,294,785"
Agincourt North,"29,113","9,120","3,929",7.41,"3,705","11,305","25,005"
...,...,...,...,...,...,...,...
Wychwood,"14,349","5,887","8,541",1.68,"1,320","6,420","11,345"
Yonge-Eglinton,"11,817","5,676","7,162",1.65,"1,225","5,860","9,995"
Yonge-St.Clair,"12,528","7,012","10,708",1.17,920,"5,960","11,170"
York University Heights,"27,593","10,170","2,086",13.23,"4,750","12,290","23,530"


In [ ]:
df_modified.drop(labels=['_id','Category','Characteristic','City of Toronto'],axis=0,inplace=True)

In [ ]:
df_modified

,"Population,2016",Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Youth (15-24 years),Working Age (25-54 years),Total - Income statistics in 2015 for the population aged 15 years and older
Agincourt North,"29,113","9,120","3,929",7.41,"3,705","11,305","25,005"
Agincourt South-Malvern West,"23,757","8,136","3,034",7.83,"3,360","9,965","20,400"
Alderwood,"12,054","4,616","2,435",4.95,"1,235","5,220","10,265"
Annex,"30,526","15,934","10,863",2.81,"3,750","15,040","26,295"
Banbury-Don Mills,"27,695","12,124","2,775",9.98,"2,730","10,810","23,410"
...,...,...,...,...,...,...,...
Wychwood,"14,349","5,887","8,541",1.68,"1,320","6,420","11,345"
Yonge-Eglinton,"11,817","5,676","7,162",1.65,"1,225","5,860","9,995"
Yonge-St.Clair,"12,528","7,012","10,708",1.17,920,"5,960","11,170"
York University Heights,"27,593","10,170","2,086",13.23,"4,750","12,290","23,530"


## Methodology

For this report I used a few different maps that could help a new investor to decide the best neighbourhood to open a coffee shop in Toronto based on it's income, population and available competitors. In order to do that I've used the 2016 Census information combined with Foursquare data to know which neighbourhood has a coffee shop in its top10 list and also those neighbourhoods which are wealthier than the rest and are well populated.


In [ ]:
df_modified.dtypes

Population,2016                                                                 object
Private dwellings occupied by usual residents                                   object
Population density per square kilometre                                         object
Land area in square kilometres                                                  object
Youth (15-24 years)                                                             object
Working Age (25-54 years)                                                       object
Total - Income statistics in 2015 for the population aged 15 years and older    object
dtype: object

In [ ]:
df_modified['Population,2016'] = df_modified['Population,2016'].str.replace(',', '').astype(int)
df_modified['Private dwellings occupied by usual residents'] = df_modified['Private dwellings occupied by usual residents'].str.replace(',', '').astype(int)
df_modified['Population density per square kilometre'] = df_modified['Population density per square kilometre'].str.replace(',', '').astype(int)
df_modified['Land area in square kilometres'] = df_modified['Land area in square kilometres'].str.replace(',', '').astype(float)
df_modified['Youth (15-24 years)'] = df_modified['Youth (15-24 years)'].str.replace(',', '').astype(int)
df_modified['Working Age (25-54 years)'] = df_modified['Working Age (25-54 years)'].str.replace(',', '').astype(int)
df_modified['Total - Income statistics in 2015 for the population aged 15 years and older'] = df_modified['Total - Income statistics in 2015 for the population aged 15 years and older'].str.replace(',', '').astype(int)

In [ ]:
df_modified.dtypes

Population,2016                                                                   int64
Private dwellings occupied by usual residents                                     int64
Population density per square kilometre                                           int64
Land area in square kilometres                                                  float64
Youth (15-24 years)                                                               int64
Working Age (25-54 years)                                                         int64
Total - Income statistics in 2015 for the population aged 15 years and older      int64
dtype: object

In [ ]:
from sklearn.cluster import KMeans

kclusters = 3

#toronto_grouped_clustering1 = df_modified.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_modified)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:500]

array([0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 2, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 2, 0, 0, 0, 1, 1, 0, 2, 1, 0,
       2, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

In [ ]:
df_modified.insert(0, 'Cluster Labels', kmeans.labels_)

#pd.set_option('display.max_rows', None)
#df_modified.reset_index(inplace=True)
#toronto_merged.drop("index",axis=1)
df_modified

,Cluster Labels,"Population,2016",Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Youth (15-24 years),Working Age (25-54 years),Total - Income statistics in 2015 for the population aged 15 years and older
Agincourt North,0,29113,9120,3929,7.41,3705,11305,25005
Agincourt South-Malvern West,0,23757,8136,3034,7.83,3360,9965,20400
Alderwood,1,12054,4616,2435,4.95,1235,5220,10265
Annex,0,30526,15934,10863,2.81,3750,15040,26295
Banbury-Don Mills,0,27695,12124,2775,9.98,2730,10810,23410
...,...,...,...,...,...,...,...,...
Wychwood,1,14349,5887,8541,1.68,1320,6420,11345
Yonge-Eglinton,1,11817,5676,7162,1.65,1225,5860,9995
Yonge-St.Clair,1,12528,7012,10708,1.17,920,5960,11170
York University Heights,0,27593,10170,2086,13.23,4750,12290,23530


In [ ]:
df_modified.loc[df_modified['Cluster Labels'] == 0]

,Cluster Labels,"Population,2016",Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Youth (15-24 years),Working Age (25-54 years),Total - Income statistics in 2015 for the population aged 15 years and older
Agincourt North,0,29113,9120,3929,7.41,3705,11305,25005
Agincourt South-Malvern West,0,23757,8136,3034,7.83,3360,9965,20400
Annex,0,30526,15934,10863,2.81,3750,15040,26295
Banbury-Don Mills,0,27695,12124,2775,9.98,2730,10810,23410
Bay Street Corridor,0,25797,15074,14097,1.83,6860,13065,23945
Bayview Village,0,21396,9532,4195,5.10,2505,10310,18730
Bedford Park-Nortown,0,23236,8607,4209,5.52,3210,8410,18560
Bendale,0,29960,10766,4011,7.47,4605,12050,24610
Birchcliffe-Cliffside,0,22291,9198,3765,5.92,2440,9075,18345
Black Creek,0,21737,7324,6282,3.46,3290,8525,16965


In [ ]:
df_modified.loc[df_modified['Cluster Labels'] == 1]

,Cluster Labels,"Population,2016",Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Youth (15-24 years),Working Age (25-54 years),Total - Income statistics in 2015 for the population aged 15 years and older
Alderwood,1,12054,4616,2435,4.95,1235,5220,10265
Bathurst Manor,1,15873,6089,3377,4.70,1940,6655,13270
Bayview Woods-Steeles,1,13154,4698,3240,4.06,1635,4490,11150
Beechborough-Greenbrook,1,6577,2650,3614,1.82,855,2750,5250
Blake-Jones,1,7727,3125,8134,0.95,885,3605,6280
...,...,...,...,...,...,...,...,...
Woodbine-Lumsden,1,7865,3449,6722,1.17,675,3790,6685
Wychwood,1,14349,5887,8541,1.68,1320,6420,11345
Yonge-Eglinton,1,11817,5676,7162,1.65,1225,5860,9995
Yonge-St.Clair,1,12528,7012,10708,1.17,920,5960,11170


In [ ]:
df_compare=df_modified.loc[df_modified['Cluster Labels'] == 2]
df_compare

,Cluster Labels,"Population,2016",Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Youth (15-24 years),Working Age (25-54 years),Total - Income statistics in 2015 for the population aged 15 years and older
Islington-City Centre West,2,43965,19328,2712,16.21,4695,20640,37410
L'Amoreaux,2,43993,15037,6144,7.16,5730,17210,37385
Malvern,2,43794,13426,4948,8.85,6620,17865,35865
Rouge,2,46496,13389,1260,36.89,6700,18510,38125
Waterfront Communities-The Island,2,65913,40756,8943,7.37,7840,45105,61985
Willowdale East,2,50434,22304,10087,5.00,6940,25850,44340
Woburn,2,53485,18436,4345,12.31,7660,21945,43410


## Result and Discussion

Based our analysis we figure out that there are several neighbourhoods which don’t have a coffee shop in its top10 list. However, not all of them are densely populated and wealthy. Islington Avenue, Malvern and Rogue are the top 3 neighbourhoods to open a coffee shop as they have less competitors in the surrounding and are wealthy and densely populated.



Negihbourhood with cluster label **2** has the **highest** population and the **highest** income. 

Negihbourhood with cluster label **1** has the **lowest** population and the **lowest** income. 

Negihbourhood with cluster label **0** has **moderate** population and **moderate** income.


Neighbourhoods with cluster label2 are the most suitables ones for opening a coffee shop

Islington Avenue doesn't have a coffee shop in its top10 most common venues.

L'Amoreaux has a coffee shop in its top6 most common venues.

Malvern and Rogue has a coffee shop in its top9 most common venues.

Willowdale has a coffee shop in its top1 most common venues.

 Woburn has a coffee shop in its top2 most common venues.

Hence from the above analysis we reach a conclusion that **Islington Avenue** , **Malvern** and **Rogue** are the most suitable venues for opening a Coffee Shop in the city of Toronto.

## Conclusion

Based our analysis we figure out that there are several neighbourhoods which don’t have a coffee shop in its top10 list. However, not all of them are densely populated and wealthy. Islington Avenue, Malvern and Rogue are the top 3 neighbourhoods to open a coffee shop as they have less competitors in the surrounding and are wealthy and densely populated.
